In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import pyjet.backend as J

import copy

import data_utils as dsb


NUM_CHANNELS = 3

PATH_TO_TRAIN = '../input/train/'
PATH_TO_TEST = '../input/test/'

NUM_TRAIN = 670
NUM_TEST = 65

In [7]:
# Load the clusters
train_cluster_ids = np.load("../clusters/train_clusters.npz")
test_cluster_ids = np.load("../clusters/test_clusters.npz")


in_set = np.vectorize(lambda a, s: a in s)

In [8]:
# Load the training data
test_ids, X_test, sizes_test = dsb.load_test_data(path_to_test=PATH_TO_TEST, img_size=None,
                                                  num_channels=NUM_CHANNELS, mode='rgb')
train_ids, X_train, Y_train = dsb.load_train_data(path_to_train=PATH_TO_TRAIN, img_size=None,
                                                  num_channels=NUM_CHANNELS, mode='rgb')
print("Number of training samples: %s" % len(train_ids))
print("X-train shape: {}".format(X_train.shape))
print("Y-train shape: {}".format(Y_train.shape))
print("X-test shape: {}".format(X_test.shape))

# Get indexes from clusters
train_clusters = np.zeros(NUM_TRAIN, dtype=int)
test_clusters = np.zeros(NUM_TEST, dtype=int)
train_clusters[in_set(train_ids, {a for a in train_cluster_ids["cluster_0"]})]= 0
train_clusters[in_set(train_ids, {a for a in train_cluster_ids["cluster_1"]})]= 1
train_clusters[in_set(train_ids, {a for a in train_cluster_ids["cluster_2"]})]= 2

test_clusters[in_set(test_ids, {a for a in test_cluster_ids["cluster_0"]})]= 0
test_clusters[in_set(test_ids, {a for a in test_cluster_ids["cluster_1"]})]= 1
test_clusters[in_set(test_ids, {a for a in test_cluster_ids["cluster_2"]})]= 2

print(train_clusters)

100%|██████████| 670/670 [00:27<00:00, 24.23it/s]

Number of training samples: 670
X-train shape: (670,)
Y-train shape: (670,)
X-test shape: (65,)
[0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 2 0 0 0 1 0 2 1 1 0 0 1 0
 0 0 0 2 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2
 0 0 1 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 2 1 0 1 0 0 0
 0 0 0 2 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 1 0 1 0 0 0 0 0 2 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 2 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 2 2 0 1 0 0 0 0 2 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 2 0 0 2 1 0 0 2 0 0
 0 0

NameError: name 'test_cluster' is not defined

In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        self.target = target.detach() * weight
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.weight = weight
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion(input * self.weight, self.target)
        self.output = input
        return self.output

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

    
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d)

Get some image loading setup

In [ ]:
# Get the style images
style_imgs = np.concatenate([X_train[train_clusters == 0], X_test[test_cluster == 0]], axis=0)